In [2]:
import cv2 as cv 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Flatten,Dropout,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
import os
main_dir = r"C:\Users\HRITHIK REDDY\Coincent Learn\AI\Mask detection\New Masks Dataset"
train_dir=os.path.join(main_dir,"Train")
test_dir=os.path.join(main_dir,"Test")
val_dir=os.path.join(main_dir,"Validation")

train_maskdir=os.path.join(train_dir,"Mask")
train_nonmaskdir = os.path.join(train_dir,"Non Mask")


In [16]:
train_masknames=os.listdir(train_maskdir)
train_nonmasknames=os.listdir(train_nonmaskdir)

In [17]:
Train_imddatagen=ImageDataGenerator(rescale=1.0/255,
                                    zoom_range=0.2,
                                    rotation_range=40,
                                    horizontal_flip=True)
Test_imddatagen=ImageDataGenerator(rescale=1.0/255)
Val_imddatagen=ImageDataGenerator(rescale=1.0/255)

Train_gen=Train_imddatagen.flow_from_directory(train_dir,
                                                target_size=(150,150),
                                                batch_size=32,
                                                class_mode='binary',
                                                )    

Test_gen=Test_imddatagen.flow_from_directory(test_dir,
                                                target_size=(150,150),
                                                batch_size=32,
                                                class_mode='binary') 

Valid_gen=Val_imddatagen.flow_from_directory(val_dir,
                                                target_size=(150,150),
                                                batch_size=32,
                                                class_mode='binary')                                                                                                          

Found 600 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [18]:
Train_gen.class_indices

{'Mask': 0, 'Non Mask': 1}

In [19]:
Train_gen.image_shape

(150, 150, 3)

In [20]:
model= Sequential()
model.add(Conv2D(32,(3,3),padding="SAME",activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 75, 75, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 37, 37, 64)       

In [21]:
model.compile(Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
history=model.fit(Train_gen,
                epochs=30,
                validation_data=Valid_gen)

Epoch 1/30
19/19 [==============================] - 20s 1s/step - loss: 1.8826 - accuracy: 0.5617 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/30
19/19 [==============================] - 18s 942ms/step - loss: 0.6376 - accuracy: 0.6200 - val_loss: 0.6705 - val_accuracy: 0.8170
Epoch 3/30
19/19 [==============================] - 18s 946ms/step - loss: 0.4612 - accuracy: 0.7983 - val_loss: 0.4701 - val_accuracy: 0.8922
Epoch 4/30
19/19 [==============================] - 18s 966ms/step - loss: 0.2854 - accuracy: 0.8967 - val_loss: 0.3476 - val_accuracy: 0.8987
Epoch 5/30
19/19 [==============================] - 18s 930ms/step - loss: 0.2852 - accuracy: 0.8983 - val_loss: 0.3648 - val_accuracy: 0.8987
Epoch 6/30
19/19 [==============================] - 18s 939ms/step - loss: 0.2899 - accuracy: 0.8983 - val_loss: 0.3578 - val_accuracy: 0.8987
Epoch 7/30
19/19 [==============================] - 18s 931ms/step - loss: 0.2459 - accuracy: 0.9033 - val_loss: 0.3003 - val_accuracy: 0.9052
Ep

In [23]:
test_loss,test_acc=model.evaluate(Test_gen)
print("Model Accuracies \n Test accuracy: {} \n Test loss: {}".format(test_acc,test_loss))


4/4 [==============================] - 2s 323ms/step - loss: 0.1524 - accuracy: 0.9500
Model Accuracies 
 Test accuracy: 0.949999988079071 
 Test loss: 0.15244804322719574


In [24]:
model.save('mask_detection_model.h5')
